# Ryzen AI CVML Library



## Goals

* Learn about the Ryzen AI CVML library features
* Run and visualize your first NPU application
* Measure the power differences running NPU and CPU/GPU backends

## References

* [Ryzen AI CVML library](https://ryzenai.docs.amd.com/en/latest/ryzen_ai_libraries.html)
* [Ryzen AI GitHub](https://github.com/amd/RyzenAI-SW/tree/main/Ryzen-AI-CVML-Library)

## 



---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT